Copyright 2023 Google LLC. SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
import torch
from PIL import Image
import pickle
import numpy as np
from IPython.display import display
from ipywidgets import HTML
from ipyevents import Event
from baukit import renormalize

from models.triplane import model_full
from utils import sky_util, soat_util_triplane, camera_util, noise_util, show


In [ ]:
torch.set_grad_enabled(False)
device = 'cuda'

# load models

In [ ]:
full_model = model_full.ModelFull('pretrained/model_triplane.pkl', 'pretrained/model_sky_360.pkl').to(device).eval()

In [ ]:
G = soat_util_triplane.init_soat_model(full_model.ground).eval().cuda()
G_pano = full_model.sky.G
grid = sky_util.make_grid(G_pano)
input_layer = G_pano.synthesis.input

# settings
fov = 60
box_warp = G.rendering_kwargs['box_warp']
G.rendering_kwargs['ray_end'] *= 2
G.rendering_kwargs['depth_resolution'] *= 2
G.rendering_kwargs['depth_resolution_importance'] *= 2
G.rendering_kwargs['y_clip'] = 8.0
G.rendering_kwargs['decay_start'] = 0.9 * G.rendering_kwargs['ray_end']
G.rendering_kwargs['sample_deterministic'] = True

# generate initial layout and skydome env map

In [ ]:
seed = 10 # np.random.randint(0, 1000)
grid_size = 5
zs, c = soat_util_triplane.prepare_zs(seed, grid_h=grid_size, grid_w=grid_size)
zs = soat_util_triplane.interpolate_zs(zs)

# generate feature planes
xz_soat = soat_util_triplane.generate_xz(zs, c) # [1, 32, 512, 512]
xy_soat = soat_util_triplane.generate_xy(zs, c) # 2 x [1, 32, 256, 512]
yz_soat = soat_util_triplane.generate_yz(zs, c) # 2 x [1, 32, 256, 512]
planes = [xy_soat, xz_soat, yz_soat]

# set up upsampler and sky inputs
z = zs[0,0] # extract a z latent for the upsampler
ws = soat_util_triplane.prepare_ws(z, torch.zeros_like(c))
sky_z = z[:, : G_pano.z_dim]

# rendered noise (may not be used depending on noise_mode for upsampler)
noise_gen = noise_util.build_soat_noise(G, grid_size)
noise_input = noise_gen.get_noise(batch_size=1, device=zs.device)

In [ ]:
# How fast we adjust. Too large and it will overshoot.
# Too small and it will not react in time to avoid mountains.
tilt_velocity_scale = .3    # Keep this small, otherwise you'll get motion sickness.
offset_velocity_scale = .5

# How far up the image should the horizon be, ideally.
# Suggested range: 0.5 to 0.7.
horizon_target = 0.65

# What proportion of the depth map should be "near" the camera, ideally.
# The smaller the number, the higher up the camera will fly.
# Suggested range: 0.05 to 0.2
near_target = 0.2

offset = 0.
tilt = 0.
initial_stabilize_frames = 10

# sample a random camera
sampled_camera, cam2world_matrix, intrinsics = soat_util_triplane.sample_random_camera(fov, box_warp, seed)
intrinsics_matrix = intrinsics[None].to(device)
# print(sampled_camera)

# balance camera above the horizon
for _ in range(10):
    adjusted_cam = camera_util.adjust_camera_vertically(sampled_camera, offset, tilt)
    outputs, horizon, near = soat_util_triplane.generate_frame(
        G, adjusted_cam, planes, ws, intrinsics_matrix, noise_input)
    # print(horizon, near)
    tilt += tilt_velocity_scale*(horizon - horizon_target)
    offset += offset_velocity_scale*(near - near_target)
print(adjusted_cam)
    
# generate sky texture
img_w_gray_sky = outputs['image_w_gray_sky']
sky_encode = full_model.sky.encode(img_w_gray_sky)
start_grid = sky_util.generate_start_grid(seed, input_layer, grid)
sky_texture = sky_util.generate_pano_transform(G_pano, sky_z, sky_encode, start_grid)
sky_texture = sky_texture.cuda()[None]
show(renormalize.as_image(sky_texture[0]))

# interactive widget

In [ ]:
l = HTML("")
h = HTML("")
d = Event(source=l, watched_events=['keydown'])
display_size = (256, 256)

camera = adjusted_cam
camera_util.INITIAL_CAMERA = adjusted_cam
    
def update_display(outputs, camera):
    composite_rgb_url = renormalize.as_url(outputs['composite'][0], size=display_size)
    

    # calculate xyz points
    ray_origins, ray_directions = G.ray_sampler(outputs['cam2world_matrix'], intrinsics_matrix, 32)
    t_val = torch.linspace(G.rendering_kwargs['ray_start'], G.rendering_kwargs['ray_end'], 100, device=device).view(1, 1, -1, 1)
    xyz = (ray_origins.unsqueeze(-2) + t_val * ray_directions.unsqueeze(-2))
    vis_rays =  camera_util.visualize_rays(G, outputs['world2cam_matrix'], xyz,
                                       xz_soat, display_size[0])
    cam_img = renormalize.as_image(vis_rays)    
    cam_url = renormalize.as_url(cam_img, size=display_size)
    img_html = ('<div class="row"> <img src="%s"/> <img src="%s"/> </div>' % (composite_rgb_url, cam_url))
    l.value = img_html
    h.value = str(camera)
    
def handle_event(event):
    global camera, offset, tilt
    camera = camera_util.update_camera(camera, event['key'], auto_adjust_height_and_tilt=True)
    c = camera_util.adjust_camera_vertically(camera, offset, tilt)
    outputs, horizon, near = soat_util_triplane.generate_frame(
        G, c, planes, ws, intrinsics_matrix, noise_input, sky_texture=sky_texture)
    # print(horizon, near)
    tilt += tilt_velocity_scale*(horizon - horizon_target)
    offset += offset_velocity_scale*(near - near_target)
    update_display(outputs, c)    

d.on_dom_event(handle_event)
display(h, l)
handle_event({'key': 'x'})